Before we start, let us first import necessary libraries to code on Python.

In [1]:
import numpy as np
import math

In [2]:
def sqrt(x):                 #Square roots generating function(Babylonian Method)
    n = 1
    for _ in range(100):
        n = (n + x/n) * 0.5
    return n

The Cumulative Distribution Function of the standard normal distribution is denoted by the $\Phi$ function:
\begin{equation}
    \Phi(x) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x_{0}} e^{-\frac{x^{2}}{2}} dx
\end{equation}

In [3]:
def my_cdf(x): #this is used to compute the cdf of a normal distribution
    return 0.5*(1+math.erf(x/sqrt(2))) #error function from math package invoked

The Asian options with geometric average; for the continuous case, is written as:
\begin{equation}
A(0,T) = e^{\frac {1}{T}\int_{0}^{T}\ln(S(t))dt}
\end{equation}

where, $S(t) = S_{0}e^{(\mu-\frac{\sigma^{2}}{2})t + \sigma W_{t}} $ and given, $W_{t}$ is a Wiener process.

In [4]:
def my_exp(x):                              #Exponential Generating Function
    return (2.71828**x)

def AsianCallOption(S0,K,r,T,sigma,N):      #Asian Call Option with geometric average
    dt = T/N                                #differential steps
    a = np.log(S0)+0.5*(r-0.5*sigma**2)*(T+dt)#Averaged first volatility terms
    b = (sigma**2)*dt+((sigma**2*(T-dt))/(6*N))*(2*N-1) # Averaged second volatility terms from Black-Scholes Eq
    x = (a/sqrt(b)) + sqrt(b)-(np.log(K)/sqrt(b))    #d1 like European Call Option
    p = my_exp(-r*T)*(my_exp(a+0.5*b)*my_cdf(x)-K*my_cdf(x-sqrt(b))) #value of Call option
    return p

Time adjusted volatility is: $\sigma = \sigma_{0} \sqrt{T}$, where $\sigma_{0}$ is historical/parametric volatility.

In [5]:
def Paths(S0,r,sigma,T,N,MC):  #This function generates different values of stock prices over time
    S = np.zeros((MC,N))       #Initialize the Stock price with zeros
    S[:,0] = S0                #Fill the first column of Stock prices data with inputs 
    dt = T/N                   #T = time frame, N = number of steps
    ft = (r-0.5*sigma**2)*dt   #First part of Black Scholes solution
    st = sigma*sqrt(dt)        #Second part of the Black Scholes solution
    wiener = np.zeros((MC,N))  #Initialize the Wiener matrix
    for k in range(N):         #Iterating over columns
        wiener[:,k] = np.random.normal(0,1,MC) #Generating random values for wiener process
    i = 0                      #initialize the while loop
    while i < MC:              #iterate the while loop number of time 'Monte Carlo Simulations' is run
        for j in range(0,N-1): #Generating 'N-1' values of stock prices over time frame
            S[i,j+1] = S[i,j]*my_exp(ft+st*wiener[i,j]) #Wiener process invoked into equation
        i = i+1
    return S                   #Generates all the Stock prices data over time frame and each steps

In [6]:
def my_max(p1,p2):                          #Maxima calculator between two numbers.
    if (p1>=p2):
        return p1
    else:
        return p2
    
def my_mean(x):                             #Mean calcutor function
    return sum(x)/len(x)

def AsianCallOptionMC(S0,K,r,T,sigma,N,MC): #Monte Carlo simulation with Asian option
    prices = np.zeros(MC)                   #Initialize payoff list
    i = 0                                   #initialize while loop
    while i < MC:                           #iterate the process for every iterations of monte carlo simulations
        path = Paths(S0,r,sigma,T,N,1)      #initialize paths value
        prices[i] = my_max(my_mean(path[0,:])-K,0)     #averaging the payoffs arithmetically for each rows
        i = i+1                                            #in Stock prices.
    price = my_mean(my_exp(-r*T)*prices)                   #payoffs is multiplied with risk-free rate
    return price                                           #returns final stock prices predicted by MC method

In [7]:
print(f"Value of Asian Call option is: {round(AsianCallOption(100,101,0.05,1,0.1,1000),2)}")
print(f"Value of Asian Call option(Monte Carlo) is: {round(AsianCallOptionMC(100,101,0.05,1,0.1,120,10000),2)}")

Value of Asian Call option is: 2.99
Value of Asian Call option(Monte Carlo) is: 2.99
